# Langchain Practice

## ChatOllama Using pre-loaded models

In [1]:
# import dependencies - general
from dotenv import load_dotenv
import os
load_dotenv('.env')
os.environ['LANGCHAIN_ENDPOINT']

'https://api.smith.langchain.com'

In [ ]:
# import dependencies - chatollama, chat templates
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage # overloaded from BaseMessage
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    ChatPromptTemplate
)

In [2]:
from langchain_ollama import ChatOllama

# grabbing the model (local, see https://github.com/ollama/ollama/blob/main/docs/api.md for full docs)
base_url = 'http://localhost:11434'
# model = 'llama3.2:latest'
model = 'sheldon'

llm = ChatOllama(
    model=model,
    base_url=base_url,
    temperature=0.8,
    num_predict=256,
    # other params ..
)

In [3]:
response = llm.invoke('hi')

response.content

"Finally, someone has deigned to acknowledge my presence. I do hope you're aware that I was in the middle of a critical experiment and had to pause it due to your interruption. Now, if you'd like to discuss the intricacies of String theory or my expertise on comic books, I'm more than happy to enlighten you. Otherwise, please, state your purpose."

## Langchain Messages
SystemMessage - pass role of LLM in run-time; can direct how the response should be generated.
\
HumanMessage - human prompt input, user query

In [6]:
from langchain_core.messages import SystemMessage, HumanMessage # overloaded from BaseMessage
llm = ChatOllama(base_url=base_url, model='llama3.2')

In [7]:
question = HumanMessage('Tell me about the Earth in 3 points.') # your prompt/question to ask
system = SystemMessage('You are an elementary school teacher. You should answer in short, easy to understand sentences.') # system description - how to respond

messages = [system, question] # feed into llm to generate response
response = llm.invoke(messages)

print(response.content)

Here are three important things to know about our planet:

1. The Earth is a big ball that floats in space and is home to all kinds of living things.
2. Our Earth has air, water, and land, which we need to survive.
3. We must take care of the Earth so it stays healthy and happy for us and future generations!


In [8]:
system = SystemMessage('You a phd professor. You should answer as such.') # showing difference caused by system prompt

messages = [system, question] 
response = llm.invoke(messages)

print(response.content)

A most intriguing inquiry! As a Ph.D. Professor of Geoscience, I'm delighted to provide you with three salient points regarding our terrestrial home, the Earth.

1. **Planetary Composition and Structure**: The Earth is a terrestrial planet composed of approximately 31% iron, 30% calcium carbonate (limestone), 10% silicates (minerals), and 6% other elements such as aluminum, magnesium, and potassium. Its crust ranges in thickness from 5-70 km, with the majority of the solid mass residing within its mantle. The Earth's core is divided into a molten outer layer (core mantle boundary) and a solid inner core, comprising approximately 20% of the planet's mass.
2. **Atmospheric Dynamics and Climate Regulation**: The Earth's atmosphere is a dynamic system that plays a crucial role in regulating our climate and supporting life. Comprising 78% nitrogen, 21% oxygen, and smaller percentages of other gases, the atmosphere is held in place by gravity and maintains its pressure through atmospheric pr

### Prompt Templates
Replaces manually writing the system message, instead can be used as a template.

In [11]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    ChatPromptTemplate
)

In [12]:
# set up templates. We can later fill in variables to provide flexibility.
system = SystemMessagePromptTemplate.from_template('You are a {school} teacher. You should answer in short, easy to understand sentences.')
question = HumanMessagePromptTemplate.from_template('Tell me about {topic} in {number} points.')

In [16]:
question.format(topic='basketball', number='3')
system.format(school='high school')

SystemMessage(content='You are a high school teacher. You should answer in short, easy to understand sentences.', additional_kwargs={}, response_metadata={})

In [17]:
messages = [system, question]
template = ChatPromptTemplate(messages)
template

ChatPromptTemplate(input_variables=['number', 'school', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are a {school} teacher. You should answer in short, easy to understand sentences.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['number', 'topic'], input_types={}, partial_variables={}, template='Tell me about {topic} in {number} points.'), additional_kwargs={})])

In [19]:
question = template.invoke({'school': 'high school', 'topic': 'basketball', 'number': '3'})

response = llm.invoke(question)
print(response.content)

Here are three key points about basketball:

1. Basketball is a team sport played on a court with two teams of five players each.
2. The objective is to score more points than the opposing team by shooting the ball into their goal, called a hoop or basket.
3. Players can move the ball by dribbling or passing it to teammates, and they have four chances, or shots, to score before the ball is turned over.


In [20]:
# showing how much faster it can be with templates:
question = template.invoke({'school': 'high school', 'topic': 'soccer', 'number': '5'})

response = llm.invoke(question)
print(response.content)

Here are five key points about soccer:

1. Soccer is a team sport played with two teams of eleven players each.
2. The objective is to score more goals than the opposing team by kicking or heading the ball into their goal.
3. Players can use any part of their body except their hands and arms to control and move the ball.
4. A match is divided into two 45-minute halves, with a 15-minute halftime break in between.
5. The team with the most goals at the end of the two halves wins the game.


# LangChain Expression Language - LCEL chains
#### For this section, please bring in previously used templates to avoid rerunning the entire notebook
Types:
- Sequential Chain
- Parallel Chain
- Router Chain
- Chain Runnables
- Custom Chain (Runnable Sequence)

In [6]:
# bringing previously used templates
from dotenv import load_dotenv
load_dotenv('.env')

from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage
base_url = 'http://localhost:11434'
llm = ChatOllama(base_url=base_url, model='llama3.2')

from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    ChatPromptTemplate
)

In [7]:
system = SystemMessagePromptTemplate.from_template('You are a {school} teacher. You should answer in short, easy to understand sentences.')
question = HumanMessagePromptTemplate.from_template('Tell me about {topic} in {number} points.')

### Sequential LCEL Chain

In [8]:
# start off the same way as using chat templates before:
messages = [system, question]
template = ChatPromptTemplate(messages)

In [9]:
# instead of creating a question using template.invoke, we chain together the template and llm
chain = template | llm # creating the chain
chain

ChatPromptTemplate(input_variables=['number', 'school', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are a {school} teacher. You should answer in short, easy to understand sentences.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['number', 'topic'], input_types={}, partial_variables={}, template='Tell me about {topic} in {number} points.'), additional_kwargs={})])
| ChatOllama(model='llama3.2', base_url='http://localhost:11434')

In [10]:
# then, we can invoke the chain together instead of two separate times (see above chat template section)
response = chain.invoke({'school': 'primary', 'number': '3', 'topic': 'poker'})
print(response.content)

I'm a teacher, not an expert on poker! But here's what I know:

* Poker is a card game where people bet with chips or money.
* Players try to make the best hand possible using cards they've been dealt.
* The person with the best hand wins all the chips in the pot.


#### Taking it a step further, we can add other things to the chain.
For example, converting the output to a text string instead of just having a response object:

In [11]:
from langchain_core.output_parsers import StrOutputParser

In [12]:
chain = template | llm | StrOutputParser()
response = chain.invoke({'school': 'college', 'number': 3, 'topic': '401k'})
print(response) # here's the main difference; can directly print the response since it'll be a string now

Here are three key points about 401(k) plans:

1. A 401(k) is an employer-sponsored retirement savings plan that allows you to contribute a portion of your paycheck to a tax-deferred account.
2. Contributions to a 401(k) are made pre-tax, reducing your taxable income for the year, and earnings grow tax-free until withdrawal.
3. Withdrawals from a 401(k) are taxed as ordinary income, typically in retirement when you're no longer working, making it a long-term savings vehicle.


## Chaining Runnables
We can use the same logic to add more things to the chain and run at once. For this example, we will take the output of the previous chain and perform analysis on it. In other words, the output (response) will be used as the input for another llm call.

In [13]:
analysis_prompt = ChatPromptTemplate.from_template('''analyze the following text: {text}
                                                   Give a rating from 1 to 5 on how difficult it is to understand. 
                                                   Provide one sentence to explain this rating.
                                                   ''')
diff_chain = chain | analysis_prompt | llm | StrOutputParser() # chain linked to analysis prompt linked to llm and str output
# diff_chain_alt = {'response': chain} | analysis_prompt | llm | StrOutputParser()
output = diff_chain.invoke({'school': 'college', 'number': 3, 'topic': '401k'}) # I can directly use the inputs from chain
print(output)

Rating: 3 out of 5

This rating indicates that the text is generally clear and concise, but may require some basic financial knowledge or understanding of tax concepts to fully comprehend. The language used is straightforward, and the three key points are easy to follow, but the text assumes a certain level of prior knowledge about retirement savings plans and taxes.


## Parallel Chains
Parallel chains are used to run multiple runnables in parallel, the final return is a dict with the result of each value under its appropriate key.

The main difference from chaining (in series) is that parallel runnables MUST BE INDEPENDENT, while series runnables can have the input of one be the output of another. For example: explain plot of story -> give rating of plot explanation -> generate star value VS. explain plot of book || explain plot of movie -> using output from both: compare which plot is better explained

In [14]:
from langchain_core.runnables import RunnableParallel

In [15]:
# start two independent chains
joke_chain = ChatPromptTemplate.from_template('tell me a joke about {topic}') | llm | StrOutputParser()
poem_chain = ChatPromptTemplate.from_template('write a {number}-line poem about {topic}') | llm | StrOutputParser()

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain) # create a parallel chain with both chains inside

# when invoking, ALL inputs need to be provided, even if they are not all shared.
output = map_chain.invoke({'topic': 'bread', 'number': 2})
print(output['joke'] + '\n==================================\n' + output['poem'])

Why did the bread go to therapy?

Because it was feeling crumby.
Softly baked, with crust so fine,
Fresh from the oven, warm and divine.


## Chain Router
Logic flow tool - uses output of a chain to affect where the chain continues. Based on the response, router can decide next chain.

Example using poem/joke chain:\
Given a user review, classify if it is positive or negative. Then, if it is positive, create a joke about the topic. Else, create a poem to console the user.

In [16]:
prompt = """Given the user review below, classify it as either being 'Positive' or 'Negative'.
            Do not respond with more than one word.

            Review: {review}
            Movie Name: {title}
"""

template = ChatPromptTemplate.from_template(prompt)

review_chain = template | llm | StrOutputParser()

review = 'This movie blew my socks off. I was at the edge of my seat from start to finish. I loved it!'

review_chain.invoke({'review': review, 'title': 'Shark Tale'})

'Positive'

In [17]:
# create a router to perform an action based on review_chain's output
def route(info):
    if 'positive' in info['sentiment'].lower():
        return joke_chain
    else:
        return poem_chain

### RunnableLambda
We will use runnablelambda to select a chain based on the output of review_chain.\
RunnableLambda(route) converts route to a runnable so that it can be linked to the chain.

In [21]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [19]:
full_chain = {'sentiment': review_chain, 'topic': lambda x: x['title'], 'number': lambda x: len(x['title']) // 5} | RunnableLambda(route) # store number in case the review is negative
full_chain

{
  sentiment: ChatPromptTemplate(input_variables=['review', 'title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review', 'title'], input_types={}, partial_variables={}, template="Given the user review below, classify it as either being 'Positive' or 'Negative'.\n            Do not respond with more than one word.\n\n            Review: {review}\n            Movie Name: {title}\n"), additional_kwargs={})])
             | ChatOllama(model='llama3.2', base_url='http://localhost:11434')
             | StrOutputParser(),
  topic: RunnableLambda(...),
  number: RunnableLambda(...)
}
| RunnableLambda(route)

Joke means positive review, poem is negative review.

In [20]:
print(full_chain.invoke({'review': review, 'title': 'Shark Tale'})) 

Why did Oscar the fish go to the party in Shark Tale?

Because he heard it was a "jaw-dropping" good time! (get it?)


### Runnable Passthrough, Runnable Lambda
For this demonstration, we'll count the char and word counts in the output.

In [23]:
def char_counts(text):
    return len(text)

def word_counts(text):
    return len(text.split())

In [27]:
lambda_chain = full_chain | StrOutputParser() | {'char_counts': RunnableLambda(char_counts), 
                                                 'word_counts': RunnableLambda(word_counts),
                                                 'full_chain_output': RunnablePassthrough()} # brings then output from full_chain as well

output = lambda_chain.invoke({'review': review, 'title': 'Shark Tale'})
print(output)

{'char_counts': 117, 'word_counts': 23, 'full_chain_output': 'Why did Oscar the fish go to the party in Shark Tale?\n\nBecause he heard it was a "jaws-dropping" good time! (get it?)'}


#### Can altnernatively perform a similar task using @chain

In [29]:
from langchain_core.runnables import chain

In [35]:
@chain # this achieves what we did before using RunnableLambda
def custom_chain(params):
    return {
        'joke': joke_chain.invoke(params),
        'poem': poem_chain.invoke(params)
    }

params = {'topic': 'baseball', 'number': 2}
output = custom_chain.invoke(params)

print(output['joke'] + '\n================\n' + output['poem'])

Why did the baseball go to the doctor?

Because it was feeling a little "off-base"! (get it?)
Here is a 2-line poem about baseball:

The crack of the bat, a symphony sweet,
Summer's magic in every swing to meet.


# Output Parsing
Exploring different forms of outputs and how we can use that information. Previously used exapmle: StrOutputParser\
Some others: JsonOutputParser, CSV Output Parser, Datetime Output Parser, Structured Output Parser (i.e. Pydantic, Json)
### First up: Pydantic Output Parser
Pydantic docs: https://docs.pydantic.dev/latest/

In [40]:
# new section, let's reimport needed dependencies
from dotenv import load_dotenv
load_dotenv('.env')

from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage
base_url = 'http://localhost:11434'
llm = ChatOllama(base_url=base_url, model='llama3.2')

from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    ChatPromptTemplate
)

In [42]:
# import pydantic required libraries
from typing import Optional
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

In [62]:
# create a pydantic class for the llm output
class Joke(BaseModel):
    """Create a joke class to act as output format, explaining what fields you expect to see in a joke output"""

    # make sure you explain this in a way your LLM can understand
    setup: str = Field(description='The setup of the joke') 
    punchline: str = Field(description='The punchline of the joke')
    rating: Optional[int] = Field(description='The rating of the joke, from 1 to 5', default=None)

In [75]:
pydantic_parser = PydanticOutputParser(pydantic_object=Joke) # create an output with Joke type
pydantic_parser

PydanticOutputParser(pydantic_object=<class '__main__.Joke'>)

In [80]:
instruction = pydantic_parser.get_format_instructions() # instructions on output format, to be given to the LLM
print(instruction)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Create a joke class to act as output format, explaining what fields you expect to see in a joke output", "properties": {"setup": {"description": "The setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "The punchline of the joke", "title": "Punchline", "type": "string"}, "rating": {"anyOf": [{"type": "integer"}, {"type": "null"}], "default": null, "description": "The rating of the joke, from 1 to 5", "title": "Rating"}}, "required": ["setup", "punchline"]}
```


In [49]:
# using this instruction 
prompt = PromptTemplate(
    template='''
    Answer the user query with a joke. Use this formatting: {format_instruction}
    Query: {query}
    Answer:
''',
    input_variables=['query'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = prompt | llm

In [50]:
output = chain.invoke({'query': 'Tell me a joke about a cat'})
output

AIMessage(content='{"setup": "Why did the cat join a band?", "punchline": "Because it wanted to be the purr-cussionist!", "rating": 4}', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-17T20:57:25.6864391Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7792276000, 'load_duration': 6065399500, 'prompt_eval_count': 313, 'prompt_eval_duration': 1090000000, 'eval_count': 37, 'eval_duration': 635000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-ef5ffe74-ddd0-4745-aacf-c70c02bb6201-0', usage_metadata={'input_tokens': 313, 'output_tokens': 37, 'total_tokens': 350})

In [51]:
print(output.content)

{"setup": "Why did the cat join a band?", "punchline": "Because it wanted to be the purr-cussionist!", "rating": 4}


In [76]:
pydantic_parser_chain = prompt | llm | pydantic_parser # using the parser this way will format your output 
output = pydantic_parser_chain.invoke({'query': 'Tell me a joke about a cat'})
print(output)

setup='Why did the cat join a band?' punchline='Because it wanted to be the purr-cussionist!' rating=None


### Parsing using .with_structured_output 
This can be much simpler than the previous method (1 line of code instead of building a whole structure). However, it will be a new llm instead of keeping an llm and designing just the output structure.

In [61]:
output = llm.invoke('Tell me a joke about a cat')
print(output.content)

Why did the cat join a band?

Because it wanted to be the purr-cussionist.


In [63]:
structured_llm = llm.with_structured_output(Joke) # pass the class into the llm as a new structured llm as an alternative to pydantic

In [65]:
structured_llm.invoke('Tell me a joke about a cat')

Joke(setup='What do you call a group of cats playing instruments?', punchline='Why did the cat join a band? Because it wanted to be the purr-cussionist!', rating=4)

### JSON Output Parser

In [66]:
from langchain_core.output_parsers import JsonOutputParser

In [72]:
json_parser = JsonOutputParser(pydantic_object=Joke) # will still need to build a Joke class as before
print(parser.get_format_instructions()) # see that it's similar to the pydantic object output

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Create a joke class to act as output format, explaining what fields you expect to see in a joke output", "properties": {"setup": {"description": "The setup of the joke", "title": "Setup", "type": "string"}, "punchline": {"description": "The punchline of the joke", "title": "Punchline", "type": "string"}, "rating": {"anyOf": [{"type": "integer"}, {"type": "null"}], "default": null, "description": "The rating of the joke, from 1 to 5", "title": "Rating"}}, "required": ["setup", "punchline"]}
```


In [77]:
# reusing the prompt template from before (with some small changes):
json_parser_chain = prompt | llm | json_parser
output = json_parser_chain.invoke({'query': 'Tell me a joke about a cat'})
print(output) # same output, but formatted as a json

{'setup': 'Why did the cat join a band?', 'punchline': 'Because it wanted to be a purr-cussionist!', 'rating': 3}


### CSV Output Parser
Can also return in csv format (comma-separated)

In [78]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

csv_parser = CommaSeparatedListOutputParser() # doesn't need a basemodel or pydantic object

print(csv_parser.get_format_instructions())

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [81]:
format_instruction = csv_parser.get_format_instructions()

csv_prompt = PromptTemplate(
    template='''
    Answer the user query with a list of values. Use this formatting: {format_instruction}
    Query: {query}
    Answer:
''',
    input_variables=['query'],
    partial_variables={'format_instruction': format_instruction}
)

csv_chain = csv_prompt | llm | csv_parser
output = csv_chain.invoke({'query': 'Generate hashtags for my instagram post. My picture is a nature picture at Yosemite National Park.'})

print(output)

['#YosemiteNationalPark', '#NatureLovers', '#WondersOfTheWorld', '#NationalParksOfUSA', '#AdventureAwaits']


### Datetime Output Parser

In [82]:
from langchain.output_parsers import DatetimeOutputParser

In [83]:
datetime_parser = DatetimeOutputParser() # also does not need a model type

datetime_instruction = datetime_parser.get_format_instructions()
print(datetime_instruction)

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 0620-11-14T03:50:08.082909Z, 1436-05-25T14:14:15.935585Z, 1879-02-14T16:59:58.247513Z

Return ONLY this string, no other words!


In [87]:
datetime_prompt = PromptTemplate(
    template='''
    Answer the user query with a datetime. Use this formatting: {format_instruction}
    Query: {query}
    Answer:
''',
    input_variables=['query'],
    partial_variables={'format_instruction': datetime_instruction}
)

datetime_chain = datetime_prompt | llm | datetime_parser
output = datetime_chain.invoke({'query': 'When did the Queen pass away?'})

print(output)

2022-09-08 18:32:00


NOTE: LLM can hallucinate with this answer, it is not always able to grab the correct information (also depends on which LLM you use and the size).

### Chat Message Memory
Saving message history for context to chat with the LLM - will achieve this by saving the generated response after running and manages this history via session_id.

We'll use a simple chain to explore this.
#### Chain *without* message history:

In [19]:
# new section, let's reimport needed dependencies
from dotenv import load_dotenv
load_dotenv('.env')

from langchain_core.messages import SystemMessage, HumanMessage
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
base_url = 'http://localhost:11434'
llm = ChatOllama(base_url=base_url, model='llama3.2')

from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder
)

template = ChatPromptTemplate.from_template('{prompt}')
chain = template | llm | StrOutputParser()

about = "My name is Sam Smith. I work for Disney."
chain.invoke({'prompt': about})

"Hi Sam, nice to meet you! It's not every day we get to talk about working for the Happiest Place on Earth - Disney!\n\nAs a Disney employee, I'm sure you have a unique perspective on the magic of the company and its impact on people all around the world.\n\nWhat's your role at Disney? Are you a cast member in one of the theme parks, a producer, or maybe something entirely different?\n\nLet's chat about all things Disney!"

In [2]:
prompt = "What is my name?"
chain.invoke({'prompt': prompt})

"I don't have any information about your name. I'm a large language model, I don't have the ability to know or retain personal information about individual users. Each time you interact with me, it's a new conversation and I don't retain any context from previous conversations.\n\nIf you'd like to share your name with me, I'd be happy to chat with you!"

As we can see, the LLM usually wouldn't keep history as context. We will now add the history.
#### Adding message history:

In [2]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [7]:
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///chat_history.db") # creates sqlite database with chat history to be used as context

In [14]:
runnable_with_history = RunnableWithMessageHistory(chain, get_session_history) # needs runnable and way to save the history

In [9]:
user_id = 'sam.smith'
history = get_session_history(user_id)

history.get_messages()
# history.clear() # clear existing messages

[]

In [18]:
runnable_with_history.invoke([HumanMessage(content=about)],
                             config={'configurable': {'session_id': user_id}})

'It seems like you\'re trying to send a message in a format that\'s typically used for API requests or other technical contexts.\n\nIf you\'d like, I can help you rephrase the message in a more conversational tone, as if it were a human reaching out:\n\n"Hi there! My name is Sam Smith and I work for Disney. Is there anything I can help you with?"'

In [20]:
runnable_with_history.invoke([HumanMessage(content="What's my name?")],
                             config={'configurable': {'session_id': user_id}})

'It looks like we have a sequence of messages here.\n\nThe first message appears to be an AIMessage, which is likely from a conversational AI system. The content suggests that Sam Smith is trying to send a message, but it\'s not in a format typically used for human-to-human communication.\n\nThe second message responds to the first message and rephrases it in a more conversational tone, making it suitable for a human conversation. It also adds some friendly language to make the response more approachable.\n\nFinally, we have a HumanMessage that asks "What\'s my name?" - which is likely an error or a test case, as this is not the context of our previous conversation.\n\nIt seems like these messages are being generated in a simulation or testing scenario, possibly to evaluate the AI system\'s ability to understand and respond to human language.'

#### Course doens't even fix this issue, not sure what he was trying to do but he just kinda moves on...
Trying to make a chatbot:

In [21]:
system = SystemMessagePromptTemplate.from_template('You are a helpful assistant.')
human = HumanMessagePromptTemplate.from_template('{input}') # new input after looking at history

messages = [system, MessagesPlaceholder(variable_name='history'), human] 

prompt = ChatPromptTemplate(messages=messages)

chain = prompt | llm | StrOutputParser()

runnable_with_history = RunnableWithMessageHistory(chain, get_session_history, 
                                                   input_messages_key='input', 
                                                   history_messages_key='history')

In [22]:
def chat_with_llm(session_id, input):
    output = runnable_with_history.invoke(
        {'input': input},
        config={'configurable': {'session_id': session_id}}
    )

    return output

In [24]:
history.clear()
user_id = 'SamSmith'
chat_with_llm(user_id, about)

"Hello Sam! Nice to meet you. It's great to know that you're part of the magical team at Disney. What brings you here today? Are you working on a new project, or perhaps looking for some assistance with an existing one? I'm all ears and ready to help in any way I can."

In [25]:
chat_with_llm(user_id, 'What is my name?')

"Your name is Sam Smith. I apologize if I didn't quite get it right earlier - as far as I know, there are several notable individuals named Sam Smith, including the British singer-songwriter Sam Smith. But without more context, it's difficult to say which one you might be! Are you the music star?"